<a href="https://colab.research.google.com/github/Ramykaz/Solar-Panel-Anomaly-Detection/blob/main/Cell_detectorbest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Step 1: Install required packages
!pip install roboflow ultralytics

# ✅ Step 2: Import necessary modules
from roboflow import Roboflow
from ultralytics import YOLO
import os
import glob

# ✅ Step 3: Download your dataset from Roboflow (YOLOv8 format with polygons)
rf = Roboflow(api_key="TYCxQVZrrdnE0E1tmaSw")
project = rf.workspace("sa-tvpln").project("cell-detector-ackxk")
version = project.version(1)
dataset = version.download("yolov8")  # Download and unzip

# ✅ Step 4: Function to convert polygon to bounding box
def polygon_to_bbox(coords):
    xs = coords[::2]
    ys = coords[1::2]
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min
    return [x_center, y_center, width, height]

# ✅ Step 5: Convert all label .txt files (train, valid, test) to bounding boxes
def convert_all_labels(base_dir):
    label_paths = [
        os.path.join(base_dir, "train", "labels"),
        os.path.join(base_dir, "valid", "labels"),
        os.path.join(base_dir, "test", "labels")
    ]

    for label_dir in label_paths:
        if not os.path.exists(label_dir):
            continue
        label_files = glob.glob(os.path.join(label_dir, "*.txt"))

        for file in label_files:
            with open(file, "r") as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = list(map(float, line.strip().split()))
                class_id = int(parts[0])
                coords = parts[1:]
                if len(coords) > 4:  # polygon
                    bbox = polygon_to_bbox(coords)
                    new_line = f"{class_id} {' '.join(map(str, bbox))}\n"
                    new_lines.append(new_line)
                else:  # already bbox
                    new_lines.append(line)

            with open(file, "w") as f:
                f.writelines(new_lines)

# ✅ Run the conversion
convert_all_labels(dataset.location)

# ✅ Step 6: Define path to the dataset config file (data.yaml)
DATA_YAML_PATH = os.path.join(dataset.location, "data.yaml")

# ✅ Step 7: Load a pretrained YOLOv8 model
model = YOLO("yolov8s.pt")  # You can switch to yolov8n.pt for faster, yolov8m.pt for better accuracy

# ✅ Step 8: Train the model
results = model.train(
    data=DATA_YAML_PATH,
    epochs=100,
    imgsz=640,
    batch=32,
    project="cell_detector_project",
    name="yolov8_cell_detector",
    patience=30,
    optimizer="SGD",
    cos_lr=True,
    fliplr=0.5,
    flipud=0.2,
    mosaic=1.0,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=5.0,
    scale=0.5,
    shear=2.0,
    perspective=0.0005,
    verbose=True
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.


Extracting Dataset Version Zip to CELL-DETECTOR-1 in yolov8:: 100%|██████████| 213/213 [00:00<00:00, 6375.14it/s]


100%|██████████| 21.5M/21.5M [00:00<00:00, 212MB/s]


Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/CELL-DETECTOR-1/data.yaml, degrees=5.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.2, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_cell_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=30, perspective=0.0005, plots=True,

100%|██████████| 755k/755k [00:00<00:00, 80.0MB/s]

...

    100/100      8.49G     0.5931      0.299     0.8582       1162        640: 100%|██████████| 3/3 [00:01<00:00,  1.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 16.66it/s]

                   all          4        315      0.971      0.994      0.977       0.81



100 epochs completed in 0.065 hours.
Optimizer stripped from cell_detector_project/yolov8_cell_detector/weights/last.pt, 22.5MB
Optimizer stripped from cell_detector_project/yolov8_cell_detector/weights/best.pt, 22.5MB

Validating cell_detector_project/yolov8_cell_detector/weights/best.pt...
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 20.29it/s]


                   all          4        315      0.973       0.99      0.977      0.817
Speed: 0.3ms preprocess, 4.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to cell_detector_project/yolov8_cell_detector
